In [18]:
import requests
import xlsxwriter
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

In [19]:
headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
          'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [20]:
base_url = 'https://hh.ru/search/resume?L_is_autosearch=false&area=113&clusters=true&exp_period=all_time&logic=phrase&no_magic=false&order_by=relevance&pos=position%2Cworkplace_organization%2Cworkplace_position&text=менеджер+по+закупкам&page='

In [29]:
def hh_pars(base_url, headers, pages):
    data = {}
    job_title_data = []
    salary_data = []
    gender_data = []
    age_data = []
    address_data = []
    job_experience_data = []
    employers_data = []
    employers_data_summery = []
    key_skills_data = []
    key_skills_data_summery = []
    education_data = []
    education_data_summery = []
    languages_data = []
    languages_data_summery = []
    
    for page in range(0,pages):
        base_url_2 = base_url + str(page) 
        #print(base_url_2)
        urls = []
        session = requests.Session()
        request = session.get(base_url_2, headers=headers)

        #Getting a list of urls with CV
        if request.status_code == 200:
            soup = bs(request.content, 'html.parser')
            divs = soup.find_all('div', attrs = {'itemscope': 'itemscope'})
            for div in divs:
                compensation = div.find('div', attrs= {'class': 'resume-search-item__compensation'}).text
                #Getting CV only with Salory
                if compensation != '':
                    try:
                        href = div.find('a', attrs= {'data-qa': 'resume-serp__resume-title'})['href']
                        href_full =  'https://hh.ru'+ href
                        urls.append(href_full)
                    except:
                        None
            #print('OK')
        else:
            print('ERROR')

        #Parsing inside urls
        for url in urls:
            session_url = requests.Session()
            request_url = session.get(url, headers=headers)
            if request_url.status_code == 200:
                soup_inside_urls = bs(request_url.content, 'html.parser')

                #job title
                job_title_searching = soup_inside_urls.find('h1')
                try:
                    job = job_title_searching.find('span').next.text
                except:
                    job = 'не указано'
                job_title_data.append(job)

                #Desired Salary
                salary_header = soup_inside_urls.find('div', attrs = {'class': 'resume-block-container'})
                salary = salary_header.find('span', attrs = {'data-qa': 'resume-block-salary'}).text
                salary_data.append(salary)

                #Gender
                blok_main_info = soup_inside_urls.find('div', attrs = {'class': 'resume-header'})
                gender = blok_main_info.find('span', attrs = {'itemprop': 'gender'}).text
                gender_data.append(gender)
                try:
                    age = blok_main_info.find('span', attrs = {'data-qa': 'resume-personal-age'}).text
                except:
                    age = np.NaN
                age_data.append(age)

                #Location
                address = blok_main_info.find('span', attrs = {'data-qa': 'resume-personal-address'}).text
                address_data.append(str.upper(address))

                #Experience
                blok_job_experience=soup_inside_urls.find('div', attrs = {'data-qa': 'resume-block-experience'}) 
                try:
                    job_experience = blok_job_experience.find('span', attrs = {'class': 'resume-block__title-text resume-block__title-text_sub'}).text
                except:
                    job_experience = np.NaN
                job_experience_data.append(job_experience)

                #Previous employers
                job_company=soup_inside_urls.find_all('div', attrs = {'itemtype': 'http://schema.org/Organization'})
                for jobs_comp in job_company:
                    obrabotka=jobs_comp.find('div', attrs = {'class': 'resume-block__sub-title'})
                    employers=obrabotka.find('span').text
                    employers_data.append(str.upper(employers))
                employers_data_summery.append(employers_data)
                employers_data = []

                #Key skills
                blok_skills=soup_inside_urls.find_all('span', attrs = {'data-qa': 'bloko-tag__text'}) 
                if blok_skills != []:
                    for key in blok_skills:
                        key_skills=key.find('span').text
                        key_skills_data.append(str.upper(key_skills))
                else:
                    key_skills_data = np.NaN
                key_skills_data_summery.append(key_skills_data)
                key_skills_data = []

                #Education
                blok_education=soup_inside_urls.find_all('div', attrs = {'data-qa': 'resume-block-education-name'}) 
                for edu in blok_education:
                    education=edu.find('span').text
                    education_data.append(str.upper(education))
                    #print(education)
                education_data_summery.append(education_data)
                education_data = []

                #Languages
                blok_languages=soup_inside_urls.find_all('p', attrs = {'data-qa': 'resume-block-language-item'}) 
                for lang in blok_languages:
                    languages=lang.contents[0]
                    try:
                        #languages_in_ru = city_translator(languages)
                        languages_data.append(str.upper(languages))
                    except:
                        languages_data = np.nan
                    #print(languages)
                languages_data_summery.append(languages_data)
                languages_data = []

                #Getting final dataset in pandas format
                data = {'Желаемая вакансия': job_title_data,
                                         'Желаемая зарплата': salary_data, 
                                         'Пол': gender_data, 
                                         'Возраст': age_data, 
                                         'Местонахождения': address_data, 
                                         'Общий стаж': job_experience_data, 
                                         'Работодатели':  employers_data_summery,
                                         'Ключевые навыки': key_skills_data_summery,
                                         'Образование': education_data_summery, 
                                         'Знание языков': languages_data_summery}    

                #Inputing list in Pandas cells
                data_pd = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in data.items()]))
               
            else:
                print('Error')
        print('.', end='')
    
    print('\n','Parsing was completed')
    return data_pd

In [31]:
data_table=hh_pars(base_url, headers, 5)

.....
 Parsing was completed


In [25]:
writer = pd.ExcelWriter('parsing_final_dataset.xlsx', engine='xlsxwriter')
data_table.to_excel(writer, 'Sheet1')
writer.save()

In [23]:
data_table

,Желаемая вакансия,Желаемая зарплата,Пол,Возраст,Местонахождения,Общий стаж,Работодатели,Ключевые навыки,Образование,Знание языков
0,Менеджер по закупкам,35 000 руб.,Женщина,36 лет,ЛИПЕЦК,Опыт работы 6 лет 4 месяца,"[ООО ОРИОН, ООО НИКА]","[ЗАКУПКИ, РАБОТА С ПОСТАВЩИКАМИ, ЗАКУПКА ТОВАР...",[],[РУССКИЙ]
1,Ведущий менеджер по закупкам,50 000 руб.,Женщина,25 лет,САНКТ-ПЕТЕРБУРГ,Опыт работы 6 лет 11 месяцев,"[РЕСТОРАН MEAT HEAD, SEA FOOD BAR НАХОДКА, ДВЕ...","[РАБОТОСПОСОБНОСТЬ, РАБОТА С ПОСТАВЩИКАМИ, ИНВ...",[ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ МОРСКОГО И РЕЧНОГ...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
2,Менеджер по закупкам..,70 000 руб.,Женщина,41 год,МОСКВА,Опыт работы 11 лет 6 месяцев,"[ПКК КАЛИБР, ООО ""ТОРГОВАЯ ДИСТРИБЬЮТОРСКАЯ КО...","[НАВЫКИ МЕЖЛИЧНОСТНОГО ДЕЛОВОГО ОБЩЕНИЯ., СПОС...",[ДАГЕСТАНСКИЙ ГОСУДАРСТВЕННЫЙ ПЕДАГОГИЧЕСКИЙ У...,[РУССКИЙ]
3,Менеджер по закупкам,40 000 руб.,Женщина,29 лет,НИЖНИЙ НОВГОРОД,Опыт работы 9 лет 3 месяца,"[ООО ИНСТРУМАВИА, ООО ИНСТРУМЕНТ, НИЖЕГОРОДСКИ...","[ЗАКЛЮЧЕНИЕ ДОГОВОРОВ, ДОГОВОРНАЯ РАБОТА, ДОКУ...",[НИЖЕГОРОДСКИЙ ГОСУДАРСТВЕННЫЙ ТЕХНИЧЕСКИЙ УНИ...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
4,Менеджер по закупкам,50 000 руб.,Мужчина,48 лет,ЛАБИНСК,Опыт работы 1 год 5 месяцев,[ООО АГРОНЕФТЕПРОДУКТ],"[СОСТАВЛЕНИЕ ДОГОВОРОВ, ТЕНДЕРЫ, ВНУТРЕННЯЯ ЛО...","[АДЫГЕЙСКИЙ ГОСУДАРСТВЕННЫЙ УНИВЕРСИТЕТ, КРАСН...",[РУССКИЙ]
...,...,...,...,...,...,...,...,...,...,...
1436,Менеджер по закупкам,50 000 руб.,Мужчина,30 лет,КОЛОМНА,Опыт работы 8 лет 11 месяцев,"[ООО ""РУСКО"", ООО ""СНЕЖНЫЙ БАРС"", ООО ""ИНТЕКС""]",NaN,[МОСКОВСКИЙ ГОСУДАРСТВЕННЫЙ ОТКРЫТЫЙ УНИВЕРСИТ...,"[РУССКИЙ, АНГЛИЙСКИЙ]"
1437,Менеджер по закупкам,35 000 руб.,Мужчина,39 лет,ОДИНЦОВО,Опыт работы 11 лет 10 месяцев,"[ФРИЛАНСЕР, ООО ""МЕДКОН"", ФРИЛАНСЕР, ООО ЛА ДЕ...","[ЗАКАЗ И ПОКУПКА ПРОДУКТОВ, ИНВЕНТАРИЗАЦИЯ., З...","[ПРОФЕССИОНАЛЬНЫЙ ЛИЦЕЙ №318, ПОВАР]",[РУССКИЙ]
1438,Менеджер по закупкам,30 000 руб.,Мужчина,36 лет,ТОЛЬЯТТИ,Опыт работы 9 лет 5 месяцев,[НОВЫЕ СТРОИТЕЛЬНЫЕ ТЕХНОЛОГИИ],NaN,"[ВОЛЖСКИЙ УНИВЕРСИТЕТ ИМ. В.Н. ТАТИЩЕВА, ТОЛЬЯ...","[РУССКИЙ, АНГЛИЙСКИЙ]"
1439,Менеджер по закупкам и логистике,45 000 руб.,Мужчина,26 лет,МОСКВА,Опыт работы 5 лет 10 месяцев,"[ФЛОРА.РОЗА, ООО ""ГАЛАС НДТ""]","[УМЕНИЕ РАБОТЬ В РЕЖИМЕ МНОГОЗАДАЧНОСТИ, ОТВЕТ...",[ТЕХНОЛОГИЧЕСКИЙ КОЛЛЕДЖ 21],[РУССКИЙ]
